In [1]:
import os
import wikipedia
from langchain.agents import tool, initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI

# Set Gemini API key
os.environ["GOOGLE_API_KEY"] = "AIzaSyCVp_"

# Load Gemini
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

# Define the tool
@tool
def symptom_to_specialist(symptom_text: str) -> str:
    """
    Given a symptom description, suggests possible conditions and specialist.
    """
    try:
        # Use Wikipedia to summarize symptoms (first word match)
        keyword = symptom_text.split()[0]  # rough search
        wiki_summary = wikipedia.summary(keyword, sentences=2)

        # Generate Gemini response
        prompt = (
            f"User described these symptoms: {symptom_text}.\n"
            f"1. What possible health condition(s) could this indicate?\n"
            f"2. Which medical specialist should they consult?\n"
            f"Be specific but user-friendly."
        )

        gemini_answer = llm.invoke(prompt)

        return f" Wikipedia Insight: {wiki_summary}\n\n Gemini Suggestion:\n{gemini_answer}"

    except wikipedia.exceptions.DisambiguationError as e:
        return f"Too many meanings found! Try being more specific. Maybe: {', '.join(e.options[:3])}"
    except wikipedia.exceptions.PageError:
        return "Wikipedia: No match found. Try simpler symptom keywords."
    except Exception as e:
        return f" Error: {str(e)}"

# Initialize agent
tools = [symptom_to_specialist]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# CLI
if __name__ == "__main__":
    print("Medical Symptom-to-Specialist Agent is Ready!")
    user_query = input("Enter your symptoms (e.g., chest pain and shortness of breath):\n> ")
    result = agent.invoke(user_query)
    print("\n Output:\n", result)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_3876\3503821017.py:45: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


Medical Symptom-to-Specialist Agent is Ready!


Enter your symptoms (e.g., chest pain and shortness of breath):
>  Fever, sore throat, and body aches




> Entering new AgentExecutor chain...
Thought: I need to determine the possible conditions and specialists based on the given symptoms.  I can use the `symptom_to_specialist` function for this.
Action: symptom_to_specialist
Action Input: "Fever, sore throat, and body aches"
Observation:  Wikipedia Insight: Fever or pyrexia in humans is a symptom of an anti-infection defense mechanism that appears with body temperature exceeding the normal range caused by an increase in the body's temperature set point in the hypothalamus. There is no single agreed-upon upper limit for normal temperature: sources use values ranging between 37.2 and 38.3 °C (99.0 and 100.9 °F) in humans.

 Gemini Suggestion:
content='The symptoms of fever, sore throat, and body aches could indicate several possibilities, ranging from common to more serious.  It\'s crucial to see a doctor for a proper diagnosis.\n\n1. **Possible Health Conditions:**\n\n* **Viral Infection (most common):** This is the most likely cause. 

In [ ]:
(Suggestions : Fever, sore throat, and body aches ,
               Chest pain and shortness of breath,
               Blurred vision and nausea,
               Joint pain and stiffness)

